In [36]:
import pickle
from matplotlib import pyplot as plt
from matplotlib import patches
import numpy as np
import matplotlib.dates as mdates
import pandas as pd
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
from sklearn.metrics import precision_recall_fscore_support  as score

pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [19]:
df = pd.read_csv('./data/loan_baseline.pnml_noise_0.15_iteration_1_seed_614_simple.csv')

In [20]:
with open('result_rf.pkl', 'rb') as fp:
    data = pickle.load(fp)

In [21]:
def anomalous_or_not(result, caseid):
    '''
    Determine following activity is anomalous or not
    If following activity is in the prediction candidates, it is normal. Otherwise, potential anomalous
    ----------
    Parameters
    result: dict
        Next activity prediction result
    caseid: str
    ----------
    Return
    anomalous_list: list
        List with each event is anomaloy or not
    '''
    anomalous_list=[]
    for x in result[caseid]:
        event = result[caseid][x]
        true_label = event[1]
        predictions = event[0]
        event_anomalous = 'Not Available'
        for y in list(predictions.values()):
            candidate_list = y[0]
            detection_result = 'Not Available'

            if candidate_list != 'Not Available':
                candidates = y[0][1]
                detection_result = 'Not Available'
                if true_label in candidates:
                    detection_result = 'Normal'
                else:
                    detection_result = 'Potential anomalous'
        event_anomalous = detection_result
        anomalous_list.append(event_anomalous)
    
    return anomalous_list

In [22]:
def anomalous_or_not_detail(result, caseid):
    '''
    Determine following activity is anomalous or not
    If following activity is in the prediction candidates, it is normal. Otherwise, potential anomalous
    ----------
    Parameters
    result: dict
        Next activity prediction result
    caseid: str
    ----------
    Return
    anomalous_list: list
        List with each event is anomaloy or not
    '''
    anomalous_list=[]
    for x in result[caseid]:
        anomalous_detail = []
        event = result[caseid][x]
        true_label = event[1]
        predictions = event[0]
        event_anomalous = 'Not Available'
        for y in list(predictions.values()):
            candidate_list = y[0]
            detection_result = 'Not Available'

            if candidate_list != 'Not Available':
                candidates = y[0][1]
                detection_result = 'Not Available'
                if true_label in candidates:
                    detection_result = 'Normal'
                else:
                    detection_result = 'Potential anomalous'
            anomalous_detail.append((detection_result, y[1]))
        anomalous_list.append(anomalous_detail)
    
    return anomalous_list

In [23]:
caseidlist = list(data.keys())
anomalous_result ={}
for caseid in caseidlist:
    anomalous_result[caseid]= anomalous_or_not_detail(data, caseid)

In [24]:
print(df.head)
print(df['noise'])

<bound method NDFrame.head of       Case ID                                Activity       Complete Timestamp      Variant  Variant index lifecycle:transition  noise
0           0                                   Start  2017-09-15 14:46:25.000   Variant 17             17                Start  Start
1           0  start_event_Loan  application received  2017-09-15 14:46:25.000   Variant 17             17             complete    NaN
2           0   Check  application  form completeness  2017-09-15 15:14:23.349   Variant 17             17             complete    NaN
3           0                    Send acceptance pack  2017-09-15 15:14:23.349   Variant 17             17                  NaN   true
4           0                       Appraise property  2017-09-15 17:15:52.828   Variant 17             17             complete    NaN
...       ...                                     ...                      ...          ...            ...                  ...    ...
8609      525            

In [32]:
true_labels = []

for pos, x in enumerate(list(df['noise'])):
    if list(df['Activity'])[pos] != 'End':
        if x == 'Start':
            x = np.nan

        true_labels.append(x)
print(len(true_labels))

8088


In [33]:
case_event_result_dict = {}
predicted_labels = []
for x in anomalous_result:
    case_event_result_dict[x] = []
    for pos, t in enumerate(anomalous_result[x]):
        case_event_result_dict[x].append([pos+1, t[0][0]])
        predicted_labels.append(t[0][0])
print(len(predicted_labels))

8088


In [52]:
total_predictions  =0
correct_prediction =0
true_label =0

true_label2=[]
predict_label2=[]
for pos,t in enumerate(predicted_labels):
    if predicted_labels[pos] != 'Not Available':
        if true_labels[pos] == 'true':
            true_label = 'Potential anomalous'
        elif np.isnan(true_labels[pos]):
            true_label = 'Normal'
        
        true_label2.append(true_label)
        predict_label2.append(predicted_labels[pos])
        if true_label == predicted_labels[pos]:
            correct_prediction +=1
        total_predictions +=1
print(correct_prediction, total_predictions)
print(correct_prediction/total_predictions)

5448 7841
0.6948093355439358


In [54]:
print(classification_report(true_label2, predict_label2))

                     precision    recall  f1-score   support

             Normal       0.86      0.77      0.81      6810
Potential anomalous       0.12      0.21      0.15      1031

           accuracy                           0.69      7841
          macro avg       0.49      0.49      0.48      7841
       weighted avg       0.77      0.69      0.73      7841



In [94]:
import pickle

threshold = [0.01,0.05,0.1,0.15,0.2,0.25]
window_size = 100

dataset = ['loan_baseline.pnml_noise_0.15_iteration_1_seed_614_sample.pkl',
'loan_baseline.pnml_noise_0.125_iteration_1_seed_27126_sample.pkl',
'loan_baseline.pnml_noise_0.09999999999999999_iteration_1_seed_14329_sample.pkl',
'loan_baseline.pnml_noise_0.075_iteration_1_seed_73753_sample.pkl',
'loan_baseline.pnml_noise_0.049999999999999996_iteration_1_seed_42477_sample.pkl',
'loan_baseline.pnml_noise_0.024999999999999998_iteration_1_seed_68964_sample.pkl']
    
    
for data in dataset:

    print('\n')
    for thr in threshold:
#         rf_name = './result/rf_thr%s_%s'%(thr, data)
#         iso_name = './result/iso_cont%s_%s'%(thr, data)
        rf_name = './result/rf_thr%s_window%s_%s'%(thr, window_size, data)
        iso_name = './result/iso_cont%s_window%s_%s'%(thr, window_size, data)


        with open(rf_name, 'rb') as f:
            rf_data = pickle.load(f)

        with open(iso_name, 'rb') as f:
            iso_data = pickle.load(f)

        rf_y_pred_avail= []
        rf_y_true_avail = []
        iso_y_pred_avail = []
        iso_y_true_avail = []

        for pos, t in enumerate(rf_data['y_pred']):
            if t != 'Not Available' and iso_data['y_pred'][pos] != 'Not Available':
                rf_y_pred_avail.append(t)
                rf_y_true_avail.append(rf_data['y_true'][pos])

                iso_y_pred_avail.append(iso_data['y_pred'][pos])
                iso_y_true_avail.append(iso_data['y_true'][pos])

#         precision, recall, fscore, support = score(y_true = rf_y_true_avail, y_pred = rf_y_pred_avail)
        precision, recall, fscore, support = score(y_true = iso_y_true_avail, y_pred = iso_y_pred_avail)
        print(fscore[1])

    #     print('----------')



0.9210192034670851
0.9149803684687406
0.903602427221753
0.894115808392592
0.8765432098765432
0.862097169965647


0.9363702682986139
0.9251566106165513
0.9112157390280814
0.8944973817495064
0.8785995623632384
0.8580163654347631


0.9432511869771648
0.9356791957043299
0.9201538640867233
0.9048696273365877
0.8856240893637688
0.8650872817955113


0.9629150495903407
0.9510683293225406
0.9361226920786182
0.9162283710648678
0.8975655285058723
0.8747609942638623


0.9728988546539765
0.963688290493676
0.9472895328503622
0.9269954851350201
0.9003063457330416
0.8696750902527076


0.9850894632206759
0.9729502688172043
0.950661853188929
0.9188470066518847
0.882099275162859
0.8464816744009849
